
<img src="img/logo-som-energia.jpg" alt="logo-som-energia" width="25%" align="center" padding="160px"/>



# PROVA TÈCNICA PROGRAMADORA I VISUALITZACIÓ DE DADES

## Som Energia

#### Àngels Villar

#### Juliol 2021

["Enunciat"](./Prova-tecnica-programadora-dades.pdf)


- Obtenir dades de l'API
    - Inicialitzacions: 
        carregar llibreries
        inicialitzar variables
        Definir funcions per a guardar/carregar dades a un fitxer
    - Agafar dades des de l'API
- Arreglar dades: creació dataframe per treballar amb més facilitat
- EDA: 
    - Anàlisi exploratòri de dades
    - Visualització dades
    - Neteja de dades, si cal
- Visualització de dades
    - Funcions visualitzacions
    - Dades últim any
    - Gràfiques
- A futur:
    - Creació mòdels predictius
    - Validació mòdel i test
    

## Obtenir dades de l'API

- [Documentació API](https://opendata.somenergia.coop/docs/)
- [OpenData](https://opendata.somenergia.coop/ui/)


In [102]:
output_notebook()

Loading BokehJS ...

### Inicialitzacions

Importem llibreries i inicialitzem variables.



In [1]:
import numpy as np
import pandas as pd
import requests
import yaml
import os


# SOM ENERGIA API
ENDPOINT_SOMENERGIA = "https://opendata.somenergia.coop/v0.2/"


# List of cities we want to access
cities = ["17114", "17010", "17109", "17133", "17149", "17183", "17162", "17185",
            "17200", "17208", "17139", "17154", "17161", "17165", "17184", "17207", "17021"]

cities_list = []


# List of metrics  we want to access
metrics = ["members", "newmembers", "canceledmembers", "contracts", "newcontracts", "canceledcontracts", "selfconsumptioncontracts",
            "newselfconsumptioncontracts", "canceledselfconsumptioncontracts"]

metrics_list = []



### Funcions auxiliars _pickle_

Els pickles son una eina que ens permet serialitzar les dades sobre les que estem treballant.

Quan tingui el dataset construit ho guardo i així no cal que estigui fent més crides a l'API.
En qualsevol moment que necessiti puc carregar ell datasete amb les dades guardades.



**Variables**

Defineixo el diirectori on guardo fitxers i quins fitxers tindré.

Em serveixen de valors per defecte a les funcions que treballen amb pickles.


In [2]:
DATASETS_PATH = "."
datos_pickle = "somenergia_data.pickle"

**Funció save_pickle**

Guarda el dataset en format __pickle__.

Arguments: 

- dir_data: directorio donde se guardan ficheros 
- pickle_file: fichero donde guardaremos el dataset 
- mydataframe: el dataframe a guardar, per defecte es df


In [3]:
def save_pickle(dir_data=DATASETS_PATH,pickle_file=datos_pickle,mydataframe=df):

    # Directori correcte?
    if not os.path.isdir(dir_data):
        print(directorio_datos,"does not exist!!")
        return print("Error!")

    # construixo on trobar fitxer
    picklefile_path = os.path.join(dir_data, pickle_file)

    # carrego dataframe al fitxer
    mydataframe.to_pickle(picklefile_path)
    return picklefile_path



NameError: name 'df' is not defined

**Funció load_pickle**


Carrego el dataframe en formato __pickle__ que he desat a disc.

Arguments:

    - dir_data: directori on deso fitxers

    - pickle_file: fitxer __pickle__ per carregar
    
Retorna dataframe


In [4]:
def load_pickle(dir_data=DATASETS_PATH,pickle_file=datos_pickle):

    # construixo on trobar fitxer
    picklefile_path = os.path.join(dir_data, pickle_file)

    # Directori correcte?
    if not os.path.isfile(pickle_file):
        print(picklefile_path,"does not exist!!")
        return print("Error!")    

    # Devolvemos el pickle
    return pd.read_pickle(picklefile_path)

### Càrrega de dades

Per cada mètrica (members, newmembers...) recullo les dades de totes les poblacions que formen la garrotxa, no he trobat grup local de la Garrotxa a què fa referència l'enunciat,  des de 1 gener de 2010.


In [5]:
# Construeixo parametres per consultar totes les ciutats de la Garrotxa
params='/by/city/monthly/from/2010-01-01?'
for c in cities:
    params+='&city='+c

In [6]:
# Per cada métrica, get dades aPI 
metrics_list = []
for metric in metrics:
    response = requests.get(ENDPOINT_SOMENERGIA + metric + params)
    print("fet response")
    dades = yaml.safe_load(response.text)
    metrics_list.append(dades['values'])

fet response
fet response
fet response
fet response
fet response
fet response
fet response
fet response
fet response


### Arreglar dades


Converteixo a dataframe per facilitar les tasques d'anàlisi de dades i visualització. També millora eficiència.

In [7]:
dates=dades['dates']
df = pd.DataFrame(list(zip(metrics_list[0],metrics_list[1],metrics_list[2],metrics_list[3],metrics_list[4],metrics_list[5],metrics_list[6],metrics_list[7],metrics_list[8])),index=dates, 
           columns =["members", "newmembers", "canceledmembers", "contracts", "newcontracts", "canceledcontracts", "selfconsumptioncontracts",
            "newselfconsumptioncontracts", "canceledselfconsumptioncontracts"])
df

,members,newmembers,canceledmembers,contracts,newcontracts,canceledcontracts,selfconsumptioncontracts,newselfconsumptioncontracts,canceledselfconsumptioncontracts
2010-01-01,0,0,0,0,0,0,0,0,0
2010-02-01,0,0,0,0,0,0,0,0,0
2010-03-01,0,0,0,0,0,0,0,0,0
2010-04-01,0,0,0,0,0,0,0,0,0
2010-05-01,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2021-04-01,366,5,1,561,14,7,31,1,0
2021-05-01,371,5,0,571,11,1,31,0,0
2021-06-01,374,3,0,576,8,3,31,0,0
2021-07-01,377,3,0,581,13,8,32,1,0


### Guardo el dataframe en _pickle_

Ara que ja tinc el conjunt de dades sobre el que vull treballa m'ho guardo en un fitxer.


In [8]:
save_pickle()

NameError: name 'save_pickle' is not defined

In [ ]:
df


### Carrego el  _pickle_ al meu dataframe

Com he estat eliminat NaN, nulls i trastejant el dataframe i penso que pot estar malmès, torno a carregar les dades originals.



In [100]:
df = load_pickle()
df

,members,newmembers,canceledmembers,contracts,newcontracts,canceledcontracts,selfconsumptioncontracts,newselfconsumptioncontracts,canceledselfconsumptioncontracts
2010-01-01,0,0,0,0,0,0,0,0,0
2010-02-01,0,0,0,0,0,0,0,0,0
2010-03-01,0,0,0,0,0,0,0,0,0
2010-04-01,0,0,0,0,0,0,0,0,0
2010-05-01,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2021-03-01,362,7,0,553,10,2,30,3,0
2021-04-01,366,5,1,561,14,7,31,1,0
2021-05-01,371,5,0,571,11,1,31,0,0
2021-06-01,374,3,0,576,8,3,31,0,0


## EDA

### Descripció de dades
Començo a analitzar les dades que acabo d'obtenir.


In [ ]:
df.tail(5)

A simple vista sense fer massa exploració de les dades, es veu alguna cosa extranya:


fila 136: 371 membres, afegeix 5, cap baixa i el següent valor


fila 137: 374?!

Mirem si ha valors nulls:

In [9]:
df.isnull().sum()

members                             0
newmembers                          0
canceledmembers                     0
contracts                           0
newcontracts                        0
canceledcontracts                   0
selfconsumptioncontracts            0
newselfconsumptioncontracts         0
canceledselfconsumptioncontracts    0
dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140 entries, 2010-01-01 to 2021-08-01
Data columns (total 9 columns):
 #   Column                            Non-Null Count  Dtype
---  ------                            --------------  -----
 0   members                           140 non-null    int64
 1   newmembers                        140 non-null    int64
 2   canceledmembers                   140 non-null    int64
 3   contracts                         140 non-null    int64
 4   newcontracts                      140 non-null    int64
 5   canceledcontracts                 140 non-null    int64
 6   selfconsumptioncontracts          140 non-null    int64
 7   newselfconsumptioncontracts       140 non-null    int64
 8   canceledselfconsumptioncontracts  140 non-null    int64
dtypes: int64(9)
memory usage: 10.9+ KB


Els valors semblen correctes, hi ha el mateix número de files en totes les columnes.

Ara mirarem les principals variables estadístiques:



In [101]:
from IPython.display import display
pd.options.display.float_format = '{:,.2f}'.format


df.describe()


,members,newmembers,canceledmembers,contracts,newcontracts,canceledcontracts,selfconsumptioncontracts,newselfconsumptioncontracts,canceledselfconsumptioncontracts
count,139.00,139.00,139.00,139.00,139.00,139.00,139.00,139.00,139.00
mean,139.83,2.77,0.06,173.37,5.28,1.13,2.79,0.24,0.01
std,113.04,2.41,0.23,182.59,5.46,1.85,7.57,0.72,0.08
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,40.00,1.00,0.00,14.50,1.00,0.00,0.00,0.00,0.00
50%,107.00,2.00,0.00,95.00,4.00,0.00,0.00,0.00,0.00
75%,227.50,4.00,0.00,298.50,9.00,1.50,0.50,0.00,0.00
max,377.00,10.00,1.00,581.00,21.00,8.00,32.00,4.00,1.00


### Anàlisi de dades

Farem servir el mòdul _ProfileReport_  de la llibreria _pandas_profiling_ per a generar un fitxer amb la descripció de les dades del dataframe.

In [18]:
from pandas_profiling import ProfileReport

fichero_profile = "reportprofile.html"

profile = ProfileReport(df, title="Profile")
profile.to_file(fichero_profile)

Summarize dataset:   0%|          | 0/23 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

**[Acces al fitxer creat per pandas_profile](./reportprofile.html)**

**Funció veure porcentatge canvis**

Para ver la variación màxima i mínima utilizamos la funcion __pct_change()__ de la libreria pandas.

Pandas dataframe.pct_change() calcula el porcentatge de canvi entre l'element actual i l'anterior, per defecte és entre la fila actual i l'anterior.


In [19]:
df_variacions = df.pct_change(fill_method ='bfill')
df_variacions.tail(10)

,members,newmembers,canceledmembers,contracts,newcontracts,canceledcontracts,selfconsumptioncontracts,newselfconsumptioncontracts,canceledselfconsumptioncontracts
2020-10-01,0.01,0.25,inf,0.01,0.29,-0.25,0.06,0.00,NaN
2020-11-01,0.01,-0.60,-1.00,0.02,0.00,-0.67,0.16,2.00,NaN
2020-12-01,0.01,1.50,NaN,0.02,0.44,3.00,0.14,0.00,NaN
2021-01-01,0.01,-0.20,NaN,0.01,-0.23,0.50,0.00,-0.67,inf
2021-02-01,0.02,0.75,inf,0.03,1.10,0.17,0.08,1.00,-1.00
2021-03-01,0.02,0.00,-1.00,0.01,-0.52,-0.71,0.11,0.50,NaN
2021-04-01,0.01,-0.29,inf,0.01,0.40,2.50,0.03,-0.67,NaN
2021-05-01,0.01,0.00,-1.00,0.02,-0.21,-0.86,0.00,-1.00,NaN
2021-06-01,0.01,-0.40,NaN,0.01,-0.27,2.00,0.00,NaN,NaN
2021-07-01,0.01,0.00,NaN,0.01,0.62,1.67,0.03,inf,NaN




Revisem estadístics bàsics que ens ajudin a fer-nos una idea de la variació de dades.


In [20]:
df_variacions.describe()

C:\Users\angels\anaconda3\lib\site-packages\numpy\lib\function_base.py:3961: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Users\angels\anaconda3\lib\site-packages\numpy\lib\function_base.py:3963: RuntimeWarning: invalid value encountered in multiply
  lerp_interpolation = asanyarray(add(a, diff_b_a*t, out=out))
C:\Users\angels\anaconda3\lib\site-packages\numpy\lib\function_base.py:3964: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)


,members,newmembers,canceledmembers,contracts,newcontracts,canceledcontracts,selfconsumptioncontracts,newselfconsumptioncontracts,canceledselfconsumptioncontracts
count,128.00,126.00,15.00,113.00,111.00,67.00,35.00,21.00,2.00
mean,inf,inf,inf,inf,inf,inf,inf,inf,inf
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.00,-1.00,-1.00,0.00,-1.00,-1.00,0.00,-1.00,-1.00
25%,0.01,-0.50,-1.00,0.01,-0.33,-0.67,0.00,-0.50,inf
50%,0.02,0.00,NaN,0.03,0.00,0.00,0.00,0.00,NaN
75%,0.03,1.00,NaN,0.05,0.93,2.25,0.15,2.00,NaN
max,inf,inf,inf,inf,inf,inf,inf,inf,inf


### Neteja dades

Elimino nulls i NaNs.



In [15]:
df_variacions.replace([np.inf, -np.inf], np.nan)

df_variacions.dropna(inplace=True)
df_variacions

,members,newmembers,canceledmembers,contracts,newcontracts,canceledcontracts,selfconsumptioncontracts,newselfconsumptioncontracts,canceledselfconsumptioncontracts
2021-02-01,0.02,0.75,inf,0.03,1.10,0.17,0.08,1.00,-1.00


### Càlcul de variacions

Cerco les dates amb els valors de variació  màxims i mínma  sobre les mètriques:

In [21]:
# Per cada métrica, mínim i màxim
variacio_metrics_min_max = []
for metric in metrics:
    variacio_metrics_min_max.append([metric,df_variacions[metric].idxmin(),df_variacions[metric].idxmax()])
variacio_metrics_min_max

[['members', datetime.date(2011, 2, 1), datetime.date(2010, 12, 1)],
 ['newmembers', datetime.date(2011, 2, 1), datetime.date(2010, 12, 1)],
 ['canceledmembers', datetime.date(2018, 7, 1), datetime.date(2018, 6, 1)],
 ['contracts', datetime.date(2012, 6, 1), datetime.date(2012, 3, 1)],
 ['newcontracts', datetime.date(2012, 6, 1), datetime.date(2012, 3, 1)],
 ['canceledcontracts', datetime.date(2014, 3, 1), datetime.date(2014, 2, 1)],
 ['selfconsumptioncontracts',
  datetime.date(2018, 10, 1),
  datetime.date(2018, 9, 1)],
 ['newselfconsumptioncontracts',
  datetime.date(2018, 10, 1),
  datetime.date(2018, 9, 1)],
 ['canceledselfconsumptioncontracts',
  datetime.date(2021, 2, 1),
  datetime.date(2021, 1, 1)]]

In [22]:
# Create the pandas DataFrame
df_variacio_metrics_min_max = pd.DataFrame(variacio_metrics_min_max, columns = ["mètrica","Mínim", "Màxim"])

# print dataframe.
df_variacio_metrics_min_max

,mètrica,Mínim,Màxim
0,members,2011-02-01,2010-12-01
1,newmembers,2011-02-01,2010-12-01
2,canceledmembers,2018-07-01,2018-06-01
3,contracts,2012-06-01,2012-03-01
4,newcontracts,2012-06-01,2012-03-01
5,canceledcontracts,2014-03-01,2014-02-01
6,selfconsumptioncontracts,2018-10-01,2018-09-01
7,newselfconsumptioncontracts,2018-10-01,2018-09-01
8,canceledselfconsumptioncontracts,2021-02-01,2021-01-01


In [23]:
df_variacio_metrics_min_max.set_index('mètrica')

,Mínim,Màxim
mètrica,,
members,2011-02-01,2010-12-01
newmembers,2011-02-01,2010-12-01
canceledmembers,2018-07-01,2018-06-01
contracts,2012-06-01,2012-03-01
newcontracts,2012-06-01,2012-03-01
canceledcontracts,2014-03-01,2014-02-01
selfconsumptioncontracts,2018-10-01,2018-09-01
newselfconsumptioncontracts,2018-10-01,2018-09-01
canceledselfconsumptioncontracts,2021-02-01,2021-01-01


## VISUALITZACIÓ DADES



###  Entorn

Preparem el que necessitem per fer les visualitzacions.

Utilitzarem la llibreria boke, amb la idea de poder construir un dashboard interactiu.


In [70]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, show, figure, show, output_file, output_notebook
from bokeh.palettes import Spectral11, colorblind, Inferno, BuGn, brewer
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource,LinearColorMapper,BasicTicker, PrintfTickFormatter, ColorBar
import datetime

METRIC="members"
DF=df

M'asseguro que lindex del detaframe té format de data

In [103]:
df.index = pd.to_datetime(df.index)

In [104]:
df.index

DatetimeIndex(['2010-01-01', '2010-02-01', '2010-03-01', '2010-04-01',
               '2010-05-01', '2010-06-01', '2010-07-01', '2010-08-01',
               '2010-09-01', '2010-10-01',
               ...
               '2020-10-01', '2020-11-01', '2020-12-01', '2021-01-01',
               '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01',
               '2021-06-01', '2021-07-01'],
              dtype='datetime64[ns]', length=139, freq=None)

In [44]:
datelist = pd.date_range(start=df.index[0],end=df.index[-1])
datelist

DatetimeIndex(['2010-01-01', '2010-01-02', '2010-01-03', '2010-01-04',
               '2010-01-05', '2010-01-06', '2010-01-07', '2010-01-08',
               '2010-01-09', '2010-01-10',
               ...
               '2021-06-22', '2021-06-23', '2021-06-24', '2021-06-25',
               '2021-06-26', '2021-06-27', '2021-06-28', '2021-06-29',
               '2021-06-30', '2021-07-01'],
              dtype='datetime64[ns]', length=4200, freq='D')

In [107]:
df['year'] = pd.DatetimeIndex(df.index).year
df['month'] = pd.DatetimeIndex(df.index).month

In [59]:
temp_df = df.groupby(['month']).mean().reset_index()
temp_df.head()

,month,members,newmembers,canceledmembers,contracts,newcontracts,canceledcontracts,selfconsumptioncontracts,newselfconsumptioncontracts,canceledselfconsumptioncontracts,year
0,1,132.25,2.50,0.00,163.67,4.17,1.17,2.42,0.17,0.08,"2,015.50"
1,2,136.50,4.33,0.08,169.50,7.67,1.83,2.67,0.25,0.00,"2,015.50"
2,3,140.25,3.75,0.00,175.83,7.33,1.00,3.00,0.33,0.00,"2,015.50"
3,4,143.42,3.25,0.08,179.83,5.67,1.75,3.42,0.42,0.00,"2,015.50"
4,5,145.92,2.50,0.00,184.83,5.58,0.67,3.67,0.25,0.00,"2,015.50"


### Funcions

**Funció plot_evolució_metrica**

Mostra evolució altes, baixes de la métrica que se li passa com a paràmetre, per defecte mostra "members"


In [149]:
def plot_evolucio_metrica(mydf=DF,mymetric=METRIC):
    # Preparo dades
    temp_df = mydf.groupby(['year', 'month']).sum().reset_index()

    # Parametre visualització
    TOOLS = 'crosshair,save,pan,box_zoom,reset,wheel_zoom'
    p = figure(title="All "+ mymetric + ", New and Canceled "+mymetric, x_axis_type='datetime', y_axis_type="linear", tools = TOOLS)

    # Contrucció plot
    all_metric = mymetric
    new_metric= "new"+mymetric
    canceled_metric= "canceled"+mymetric
    
    p.line(df.index, df[all_metric], legend_label=mymetric, line_color="purple", line_width = 3)
    p.line(df.index, df[new_metric], legend_label="New "+ mymetric, line_color="blue", line_width = 3)
    p.line(df.index, df[canceled_metric], legend_label="Canceled "+mymetric, line_color = 'coral', line_width = 3)
    p.legend.location = "top_left"
    p.xaxis.axis_label = 'Year'
    p.yaxis.axis_label = 'Count'
    
    return(p)


**Funció plot_mapa_calor**

Mostra mapa de calor data de la métrica que se li passa com a paràmetre, per defecte mostra "members"


In [150]:
def plot_mapa_calor(mydf=DF,mymetric=METRIC):
    # Preparo dades
    temp_df = mydf.groupby(['year', 'month']).sum().reset_index()
 
    # Parametre visualització
    TOOLS = 'crosshair,save,pan,box_zoom,reset,wheel_zoom'
    hm = figure(title="Year Month "+ mymetric, tools=TOOLS, toolbar_location='above')

    # Contrucció plot
    source = ColumnDataSource(temp_df)
    colors = brewer['BuGn'][9]
    colors = colors[::-1]
    mapper = LinearColorMapper(
        palette=colors, low=temp_df[mymetric].min(), high=temp_df[mymetric].max())
    hm.rect(x="year", y="month",width=2,height=1,source = source,  
        fill_color={
            'field': mymetric,
            'transform': mapper
        },
        line_color=None)
    color_bar = ColorBar(
        color_mapper=mapper,
        major_label_text_font_size="10pt",
        ticker=BasicTicker(desired_num_ticks=len(colors)),
        formatter=PrintfTickFormatter(),
        label_standoff=6,
        border_line_color=None,
        location=(0, 0))

    hm.add_layout(color_bar, 'right')
    hm.xaxis.axis_label = 'Year'
    hm.yaxis.axis_label = 'Month'

    return(hm)


**Funció plot_avg_barres**

Mostra visualitzador mitjana métrica que se li passa com a paràmetre, per defecte mostra "members"


In [151]:
def plot_avg_barres(mydf=DF,mymetric=METRIC):
    # Preparo dades
    temp_df = mydf.groupby(['year', 'month']).mean().reset_index()
 
    # Construeixo visualitzatció
    TOOLS = 'crosshair,save,pan,box_zoom,reset,wheel_zoom'
    p = figure(plot_height=350,
        title="AVG "+mymetric,
        toolbar_location='above')

    # Contrucció plot
    p.vbar(x=temp_df.month, top=temp_df[mymetric], width=0.9)

    p.y_range.start = 0
    p.x_range.range_padding = 0.1
    p.xgrid.grid_line_color = None
    p.axis.minor_tick_line_color = None
    p.outline_line_color = None
    p.xaxis.axis_label = 'Month'
    p.yaxis.axis_label = 'AVG '+mymetric   
    
    return(p)


### Dades darrer any

In [148]:
# agafo dades del darrer any
last_year = df.loc[pd.to_datetime('2020-01-01'):pd.to_datetime('2021-01-01')]

<ipython-input-148-c33aba2a71b7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_year['year'] = pd.DatetimeIndex(last_year.index).year
<ipython-input-148-c33aba2a71b7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_year['month'] = pd.DatetimeIndex(last_year.index).month


**Gràfica Evolució mètrica**

In [152]:
output_notebook()
for m in ["members","contracts","selfconsumptioncontracts"]:
    show(plot_evolucio_metrica(last_year,m))


**Gràfica plot_mapa_calor**

In [152]:
output_notebook()
for m in ["members","contracts","selfconsumptioncontracts"]:
    show(plot_mapa_calor(last_year,m))

 **Gràfica plot_avg_barres**

In [1]:
output_notebook()
for m in ["members","contracts","selfconsumptioncontracts"]:
    show(plot_avg_barres(last_year,m))

NameError: name 'show' is not defined